## This document performs the data preprocessing of toronto yelp data 

In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize
import yaml
import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

### Load Toronto Data, review as df_toronto_review and business as df_business
#### (Import Toronto data as df,  userJson as df_user)

In [2]:
# read csv data reviews 
df_toronto_review = pd.read_csv('../data/toronto_reviews.csv')

# read business csv data 
df_business = pd.read_csv('../data/businesses_final_toronto.csv')

In [3]:
df_toronto_review.head(2)

,Unnamed: 0,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,rating,...,review_language,review_text,ufc,user_id,user_loc,vote_count,Updated,Year,Month,Day
0,0,0,bSQGCheX1BwvL67Az1OJlA,90,False,[],[],False,231.0,5.0,...,en,I had a good experience at this store. I was g...,"[10, 1, 10]",q41qqkChL9fRxF43cnkgbw,"Houston, TX",11.0,False,2018,5,9
1,1,1,bSQGCheX1BwvL67Az1OJlA,0,False,[],[],False,9.0,1.0,...,en,I reaaaaaally wish I had read all the stories ...,"[0, 0, 0]",tNT5I9-QcljS-7w_doXCXQ,"Toronto, Canada",0.0,False,2018,6,3


In [4]:
len(df_toronto_review)

747183

In [5]:
df_toronto_review.user_id.unique().size

128420

In [6]:
# rename business id
df_business.rename(columns={'id': 'business_id'}, inplace=True)

In [7]:
df_business.head(2)

,Unnamed: 0,alias,categories,coordinates,display_phone,distance,business_id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,0,the-ossington-toronto,"[{'alias': 'divebars', 'title': 'Dive Bars'}]","{'latitude': 43.6456, 'longitude': -79.41903}",+1 416-850-0161,4831.221092,KUdj08tVC3tvIT0rM6V8uQ,https://s3-media1.fl.yelpcdn.com/bphoto/dEQPAb...,False,"{'address1': '61 Ossington Avenue', 'address2'...",The Ossington,1.416850e+10,$$,3.0,46,[],https://www.yelp.com/biz/the-ossington-toronto...
1,1,life-is-sweet-toronto-3,"[{'alias': 'desserts', 'title': 'Desserts'}, {...","{'latitude': 43.67212500034, 'longitude': -79....",+1 416-698-0555,1716.906703,rnjFHvb1I-KlnGvv_o_Zxg,https://s3-media2.fl.yelpcdn.com/bphoto/s11QxZ...,False,"{'address1': '2345 Queen Street E', 'address2'...",Life is Sweet,1.416698e+10,$$,4.0,10,[],https://www.yelp.com/biz/life-is-sweet-toronto...


In [8]:
print("total business numbers:", len(df_business))

total business numbers: 32123


## Checking all unique categories

In [9]:
import re

In [10]:
df_business.categories.unique().size

7197

In [11]:
categories_list = list(df_business.categories.unique())

In [12]:
categories_list[65]

"[{'alias': 'acupuncture', 'title': 'Acupuncture'}, {'alias': 'reflexology', 'title': 'Reflexology'}, {'alias': 'massage_therapy', 'title': 'Massage Therapy'}]"

In [13]:
res = re.findall(r'\w+', categories_list[65]) 

In [14]:
print ("The list of words is : " +  str(res)) 

The list of words is : ['alias', 'acupuncture', 'title', 'Acupuncture', 'alias', 'reflexology', 'title', 'Reflexology', 'alias', 'massage_therapy', 'title', 'Massage', 'Therapy']


In [15]:
res[2] != 'title'

False

In [16]:
new_categories_list = []
for category in categories_list:
    res = re.findall(r'\w+', category)
    for word in res:
        if (word != 'alias') and (word != 'title'):
            new_categories_list.append(word)

In [17]:
set(new_categories_list)

{'Flowers',
 'Fur',
 'salad',
 'pilates',
 'Bed',
 'Sharpening',
 'oralsurgeons',
 'Lawyers',
 'Oil',
 'metrostations',
 'Tiki',
 'Developers',
 'cantonese',
 'Mex',
 'Soup',
 'Persian',
 'soulfood',
 'portuguese',
 'Decorations',
 'osteopathicphysicians',
 'paydayloans',
 'ayurveda',
 'Kebab',
 'electricians',
 'Patio',
 'Training',
 'Lessons',
 'hotdogs',
 'sleepspecialists',
 'candy',
 'Roasteries',
 'virtualrealitycenters',
 'Sunglasses',
 'Pediatric',
 'hair_extensions',
 'Throat',
 'Radio',
 'Champagne',
 'mosques',
 'musicinstrumentservices',
 'pediatricians',
 'Up',
 'cookingclasses',
 'vehiclewraps',
 'Rafting',
 'rafting',
 'laotian',
 'Passport',
 'Haunted',
 'Supplies',
 'antiques',
 'Television',
 'Walkers',
 'goldbuyers',
 'Climbing',
 'Airport',
 'buildingsupplies',
 'creperies',
 'Threading',
 'transmissionrepair',
 'wine_bars',
 'bodycontouring',
 'Party',
 'British',
 'Nutritionists',
 'cardiology',
 'Vape',
 'importedfood',
 'swimminglessons',
 'Loan',
 'Marketing',


## Data Processing

#### High level processsing

In [18]:
# keywords used to identify restaurants
restaurantString = "restaurant|Bar|Bakeries|food|Coffee|Tea|Waffles|Desserts\
|sandwiches|Cupcakes|cafes|Breakfast|Brunch|Lunch|Dinner|chicken|wings\
|singaporean|malaysian|indonesian|russian|taiwanese|american|vietnamese|moroccan\
|cantonese|caribbean|mideastern|asian|italian|korean|himalayan|african|nepalese|laotian|sushi|hawaiian\
|mediterranean|caterers|fish|chips|fishnchips|thai|salad|chinese|seafood|irish|portuguese|french|vegan|raw_food\
|pizza|latin|lebanese|burgers|gluten-free|japanese|pakistani|mexican|sushi|argentine|lounges|barbeque\
|canadian|dimsum|halal|afghan|donuts|juice|smoothies|bagels|ramen|icecream|pubs|falafel\
|noodles|soup|cheese|colombian|turkish|ethiopian|peruvian|hungarian|ukrainian|indian\
|persian|brazilian|vegetarian|austrian|arabian|haitian|cambodian|german|greek|Barbeque|Buffets\
|Cajun/Creole|Cajun|Cuban|Czech|Delis|Diners|Filipino|Festivals|Hot Dogs|Hot Pot|Macarons|Modern European\
|Nightlife|Salvadoran|Venezuelan|tapas|australian|kosher|wine|creperies|cafe|bangladeshi|cakes|brasseries\
|cideries|smokehouse|patisserie|hindu|hong kong|hakka|veggies|plates|hkcafe|spanish|grilling"

# We only take restaurant business with the above keywords
cond_category_restaurant = df_business["categories"].str.contains(restaurantString,case=False)

#df_business_Restaurant = df_business[cond_category_restaurant]
df_matched_business = df_business[cond_category_restaurant]

# keywords used to identify businesses we want to filter out
filterString = "acupuncture|grocery|bookstores|musicvideo|yoga|servicestations|convenience|homeappliancerepair\
|appliances|Repair|bike|repair|Used|toys|wholesale|supplies|hair|barbers|Hair Salons|Archery|Adult Entertainment|Amateur Sports Teams\
|Airport Lounges|Vinyl Records|Pilates|fitness|Real Estate Agents|Real Estate Services|Recreation Centers|Reflexology\
|massage|Tattoo|Traditional Chinese Medicine|Trainers|Walking Tours|Virtual Reality Centers|Musical Instrument Services|Laser Tag\
|Community Service/Non-Profit|Flea Markets|Tours|Golf|Hotels|Kids Activities|Accessories|Bartending Schools|audio|foodbanks\
|barreclasses|bartenders|kitchenandbath|cosmetics|Bed & Breakfast|bowling|boxing|teambuilding|casinos|Centers|movietheaters\
|musicalinstrumentsandteachers|colleges|couriers|Customized Merchandise|gyms|escapegames|videogamestores|gift|goods|laundromat\
|martial|muaythai|eventplanning|Performing Arts|personalchefs|pets|piano|sportsteams|schools|spas|tobaccoshops|vape|wedding_planning\
|winetours|beer_and_wine|Wineries|hvac|winetastingroom|hindu_temples|fireplace|artclasses|Arcades|galleries|Personal Shopping|Strip Clubs\
|grillingequipment|herbsandspices|bubbletea"

df_business_Restaurant = df_matched_business[~df_matched_business["categories"].str.contains(filterString,case=False)]

print('Matched categories, before filtering size: ', len(df_matched_business))
print('Matched categories, after filtering size: ', len(df_business_Restaurant))

Matched categories, before filtering size:  10422
Matched categories, after filtering size:  9106


#### Exclude Catering Category

In [19]:
df_business_Restaurant = df_business_Restaurant[~(df_matched_business["price"].isna() & \
                                               df_matched_business["categories"].str.contains("cater",case=False))]
print('Matched categories, after filtering size: ', len(df_business_Restaurant))

Matched categories, after filtering size:  8972


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


#### Exclude Food Deliver Service

In [20]:
df_business_Restaurant = df_business_Restaurant[~((df_matched_business["review_count"] <= 6) & \
                                                  df_matched_business["categories"].str.contains("fooddeliveryservices",case=False))]
                                    
print('Matched categories, after filtering size: ', len(df_business_Restaurant))

Matched categories, after filtering size:  8933


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


#### Filter out specific restaurants

In [21]:
filtered_out_restaurant = "8090-ktv-toronto|soho-ktv|prestige-by-night-toronto|hard-luck-bar|ktv|mimico-pub-and-grill|bar-karaoke-lounge\
                            |lighthouse-shisha-lounge|arabica-lounge|Purdy|dried-nuts-and-fruits|tangs-company|the-painted-lady|baby-g\
                            |Coalition|bassline-music-bar|underground-garage|cadillac-lounge|internet-cafe|super-tech|netconnect-cafe|invictus\
                            |gigabites|internet|penta-q|freshly-squeezed-toronto"

In [22]:
df_business_Restaurant = df_business_Restaurant[~(df_matched_business["alias"].str.contains(filtered_out_restaurant,case=False))]
                                    
print('Matched categories, after filtering size: ', len(df_business_Restaurant))

Matched categories, after filtering size:  8903


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


#### Adding back specific restaurants

In [23]:
include_restaurant = "tasty-pot|pho-saigon-restaurant|kin-kin-bakery-and-bubble-tea|hk-sweets|mika-sushi\
                      |next-station|green-grotto"

In [24]:
df_business_Restaurant = df_business_Restaurant.append(df_matched_business[(df_matched_business["alias"].str.contains(include_restaurant,case=False))])
                                    
print('Matched categories, after filtering size: ', len(df_business_Restaurant))

Matched categories, after filtering size:  8919


#### Check review number

In [25]:
# df_business_Restaurant = df_business_Restaurant[~(df_matched_business["review_count"] < 8)]
                                    
# print('Matched categories, after filtering size: ', len(df_business_Restaurant))

### Check the remaining categories

In [26]:
categories_list_final1 = list(df_business_Restaurant.categories.unique())
new_unique_categories_list = []
for category in categories_list_final1:
    res = re.findall(r'\w+', category)
    for word in res:
        if (word != 'alias') and (word != 'title'):
            new_unique_categories_list.append(word)

In [27]:
set(new_unique_categories_list)

{'Acai',
 'Afghan',
 'African',
 'American',
 'Antiques',
 'Arabian',
 'Argentine',
 'Asian',
 'Australian',
 'Austrian',
 'Bagels',
 'Bakeries',
 'Bangladeshi',
 'Bar',
 'Barbeque',
 'Bars',
 'Beer',
 'Belgian',
 'Bistros',
 'Blues',
 'Bowls',
 'Brasseries',
 'Brazilian',
 'Breakfast',
 'Breweries',
 'Brewpubs',
 'British',
 'Brunch',
 'Bubble',
 'Buffets',
 'Burgers',
 'Burmese',
 'Butcher',
 'Cafe',
 'Cafes',
 'Cajun',
 'Cake',
 'Cakes',
 'Cambodian',
 'Canadian',
 'Candy',
 'Cannabis',
 'Cantonese',
 'Caribbean',
 'Caterers',
 'Champagne',
 'Cheese',
 'Cheesesteaks',
 'Chicken',
 'Chinese',
 'Chips',
 'Chocolatiers',
 'Cideries',
 'Classes',
 'Clinics',
 'Clubs',
 'Cocktail',
 'Coffee',
 'Colombian',
 'Comedy',
 'Comfort',
 'Cooking',
 'Court',
 'Cream',
 'Creole',
 'Creperies',
 'Cuban',
 'Cupcakes',
 'Custom',
 'Czech',
 'Dance',
 'Delicatessen',
 'Delis',
 'Delivery',
 'Desserts',
 'Dim',
 'Diners',
 'Dinner',
 'Dive',
 'Do',
 'Dogs',
 'Donairs',
 'Donuts',
 'Eastern',
 'Egyptia

### Find if there are any not counted restaurant

In [28]:
#Checking the filtered out businesses at the first step to see if missed any restaurants
cond_category_Not_restaurant = ~df_business["categories"].str.contains(restaurantString,case=False)
business_not_res = df_business[cond_category_Not_restaurant]
# for index, value in business_not_res['categories'].items():
#     print("Index : {}, Value : {}".format(index, value))

In [29]:
df_cond_category_Not_restaurant = df_business[~df_business["categories"].str.contains(restaurantString,case=False)]
df_cond_category_Not_restaurant_2 = df_cond_category_Not_restaurant[~df_cond_category_Not_restaurant["categories"].str.contains(filterString,case=False)]

In [30]:
df_cond_category_Not_restaurant_2.head(1)

,Unnamed: 0,alias,categories,coordinates,display_phone,distance,business_id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
4,4,locksmith-vaughan-vaughan-3,"[{'alias': 'locksmiths', 'title': 'Keys & Lock...","{'latitude': 43.7975922, 'longitude': -79.4397...",+1 416-477-5464,17551.531314,Smw9D566sGyeatfttpzb7Q,https://s3-media3.fl.yelpcdn.com/bphoto/Xpj30m...,False,"{'address1': '', 'address2': None, 'address3':...",Locksmith Vaughan,1.416478e+10,NaN,5.0,1,[],https://www.yelp.com/biz/locksmith-vaughan-vau...


In [31]:
categories_list_final2 = list(df_cond_category_Not_restaurant_2.categories.unique())
new_unique_categories_list = []
for category in categories_list_final2:
    res = re.findall(r'\w+', category)
    for word in res:
        if (word != 'alias') and (word != 'title'):
            new_unique_categories_list.append(word)

In [32]:
print(set(new_unique_categories_list))

{'Fur', 'oralsurgeons', 'Lawyers', 'Oil', 'metrostations', 'Developers', 'Mex', 'Decorations', 'osteopathicphysicians', 'paydayloans', 'ayurveda', 'electricians', 'Patio', 'Training', 'sleepspecialists', 'candy', 'Sunglasses', 'Pediatric', 'Throat', 'Radio', 'mosques', 'pediatricians', 'cookingclasses', 'vehiclewraps', 'Rafting', 'rafting', 'Passport', 'Haunted', 'antiques', 'Television', 'Walkers', 'goldbuyers', 'Climbing', 'Airport', 'Threading', 'bodycontouring', 'Party', 'Nutritionists', 'cardiology', 'Loan', 'Marketing', 'astrologers', 'Crafts', 'Psychiatrists', 'buddhist_temples', 'fireworks', 'Consulting', 'decksrailing', 'Decks', 'hospitals', 'Clothing', 'auto_detailing', 'hottubandpool', 'duilawyers', 'First', 'massmedia', 'Riding', 'Stations', 'Doctors', 'Auction', 'Sewing', 'rehabilitation_center', 'Share', 'Officiants', 'Motor', 'Urgent', 'Stucco', 'airport_shuttles', 'Pharmacy', 'educationservices', 'commissionedartists', 'mobilephones', 'Systems', 'Control', 'homehealthca

### Run this cell to check filtered out restaurants

In [33]:
# cat_set = []
# for cat in tqdm(business_not_res.categories.values):
#     temp = yaml.safe_load(cat)
# #     ans.append([temp[i]["title"] for i in range(len(temp))])
    
#     #loop through key, append title as categories 
#     for i in tqdm(range(len(temp))):
#         cat_set.append(temp[i]["title"])
# set(cat_set)

In [42]:
#Checking invalid businesses
#pd.options.display.max_colwidth = 90  
#set(df_business_Restaurant[df_business_Restaurant["categories"].str.contains('Qi Gong',case=False)].categories)

#checking original missed restaurants
#Czech
#df_business[df_business["categories"].str.contains('Venezuelan',case=False)]

In [43]:
# for index in range(0,100):
#     print(business_not_res['categories'][index])
#business_not_res['categories']

# for index, value in business_not_res['categories'].items():
#     print("Index : {}, Value : {}".format(index, value))

In [44]:
#Checking business name
# df_business['name'][31600]

In [34]:
# Drop duplicate values in df_business
print(len(df_business_Restaurant))
df_business_clean = df_business_Restaurant.drop_duplicates(subset ="business_id", 
                     keep = False)
print(len(df_business_clean))
df_business_clean = df_business_clean.reset_index()

8919
8907


## Separate out longitude and altitude

In [35]:
latitudeList = []
longitudeList = []

for business in range(len(df_business_clean)):
    latitude= yaml.safe_load(df_business_clean.coordinates[business])['latitude']
    longitude = yaml.safe_load(df_business_clean.coordinates[business])['longitude']
    latitudeList.append(latitude)
    longitudeList.append(longitude)

In [36]:
df_latitude = pd.DataFrame(np.asarray(latitudeList))
df_latitude.rename(columns={0: "latitude"}, inplace = True)
df_longitude = pd.DataFrame(np.asarray(longitudeList))
df_longitude.rename(columns={0: "longitude"}, inplace = True)

In [37]:
df_business_clean["latitude"] = df_latitude
df_business_clean["longitude"] = df_longitude
df_business_clean = df_business_clean.drop(df_business_clean[df_business_clean['longitude'] == 'None'].index)
df_business_clean = df_business_clean.drop(df_business_clean[df_business_clean['latitude'] == 'None'].index)
df_business_clean = df_business_clean.reset_index()

## Changing the format of category

In [38]:
# Rename Month as Day and Day as Month(error occurs on original data)
df_toronto_review=df_toronto_review.rename(columns = {'Month':'new_Day'})
df_toronto_review=df_toronto_review.rename(columns = {'Day':'Month'})
df_toronto_review=df_toronto_review.rename(columns = {'new_Day':'Day'})

### Merging review and business data 

In [39]:
# inner join two full dataset
df = pd.merge(df_toronto_review, df_business_clean, on='business_id', how='inner')

In [40]:
df.shape

(493424, 43)

In [41]:
# merge the date and turn it into string
df['date'] = pd.to_datetime(df[['Year', 'Month','Day']])
df['date'] = df['date'].astype(str)

In [42]:
# rename the columns to match the algorithm 
df.rename(columns={'rating_x': 'stars','rating_y': 'business_stars','review_text': 'text',}, inplace=True)

In [43]:
# get rid of the rows without user_id
df = df[~df["user_id"].isnull()]

In [44]:
pd.set_option('display.max_columns', None)
df.head(2)

,Unnamed: 0_x,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,stars,review_count_x,review_date,review_id,review_language,text,ufc,user_id,user_loc,vote_count,Updated,Year,Day,Month,level_0,index,Unnamed: 0_y,alias,categories,coordinates,display_phone,distance,image_url,is_closed,location,name,phone,price,business_stars,review_count_y,transactions,url,latitude,longitude,date
5,65,65,oRRCe6RruoHE1nMfTlMREA,0,False,[],[],False,3.0,4.0,3,"['9', '14', '2018']",dIaFxaGjQwrAAnKm57Se_w,en,"Cute location, however it is quite small. The ...","[0, 0, 0]",0HVBYtQOhI8sVc7n4bZxCw,"Mississauga, Canada",0.0,False,2018,14,9,1902,8511,8511,black-goat-cafe-toronto,"[{'alias': 'cafes', 'title': 'Cafes'}]","{'latitude': 43.59672, 'longitude': -79.52254}",+1 416-792-7200,5179.05848,https://s3-media2.fl.yelpcdn.com/bphoto/QWleI6...,False,"{'address1': '3261 Lake Shore Boulevard W', 'a...",Black Goat Cafe,1.416793e+10,$,3.5,17,[],https://www.yelp.com/biz/black-goat-cafe-toron...,43.59672,-79.52254,2018-09-14
6,66,66,oRRCe6RruoHE1nMfTlMREA,12,False,[],[],False,NaN,4.0,8,"['5', '15', '2018']",ruTcyqV1E7UlOZ4I2ZLJHQ,en,"Pulled pork burger, salmon and cream cheese ba...","[1, 0, 0]",Qr5QmYRZHrh7zaqjYw63AQ,"Toronto, Canada",1.0,False,2018,15,5,1902,8511,8511,black-goat-cafe-toronto,"[{'alias': 'cafes', 'title': 'Cafes'}]","{'latitude': 43.59672, 'longitude': -79.52254}",+1 416-792-7200,5179.05848,https://s3-media2.fl.yelpcdn.com/bphoto/QWleI6...,False,"{'address1': '3261 Lake Shore Boulevard W', 'a...",Black Goat Cafe,1.416793e+10,$,3.5,17,[],https://www.yelp.com/biz/black-goat-cafe-toron...,43.59672,-79.52254,2018-05-15


In [45]:
df = df.reset_index(drop=True)

In [46]:
dfff = df.copy()

change category format

In [47]:
#A list of categories
ans = []
for cat in tqdm(df.categories.values):
    temp = yaml.safe_load(cat)
#     ans.append([temp[i]["title"] for i in range(len(temp))])
    cat_str = ""
    #loop through key 
    for i in range(len(temp)):
        #if first element
        if i == 0:
            cat_str += temp[i]["title"]
            #if more than one title
            if len(temp) >1:
                cat_str += ", "
        #if last element
        elif i == len(temp)-1:
            cat_str += temp[i]["title"]
        else:
            cat_str += temp[i]["title"]
            cat_str += ", "
    ans.append(cat_str)
    

100%|██████████| 402927/402927 [03:38<00:00, 1841.79it/s]


In [48]:
arr_temp = np.asarray(ans)

In [49]:
df_cat = pd.DataFrame(arr_temp)
df_cat.rename(columns={0: "cat"}, inplace = True)

In [50]:
#Create a new column for the review df
df["cat"] = df_cat

In [51]:
pd.set_option('display.max_rows', 100000)

In [52]:
#Delete original categories column
del df['categories']

In [53]:
df.rename(columns={"cat": "categories"}, inplace = True)

In [54]:
df.head(2)

,Unnamed: 0_x,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,stars,review_count_x,review_date,review_id,review_language,text,ufc,user_id,user_loc,vote_count,Updated,Year,Day,Month,level_0,index,Unnamed: 0_y,alias,coordinates,display_phone,distance,image_url,is_closed,location,name,phone,price,business_stars,review_count_y,transactions,url,latitude,longitude,date,categories
0,65,65,oRRCe6RruoHE1nMfTlMREA,0,False,[],[],False,3.0,4.0,3,"['9', '14', '2018']",dIaFxaGjQwrAAnKm57Se_w,en,"Cute location, however it is quite small. The ...","[0, 0, 0]",0HVBYtQOhI8sVc7n4bZxCw,"Mississauga, Canada",0.0,False,2018,14,9,1902,8511,8511,black-goat-cafe-toronto,"{'latitude': 43.59672, 'longitude': -79.52254}",+1 416-792-7200,5179.05848,https://s3-media2.fl.yelpcdn.com/bphoto/QWleI6...,False,"{'address1': '3261 Lake Shore Boulevard W', 'a...",Black Goat Cafe,1.416793e+10,$,3.5,17,[],https://www.yelp.com/biz/black-goat-cafe-toron...,43.59672,-79.52254,2018-09-14,Cafes
1,66,66,oRRCe6RruoHE1nMfTlMREA,12,False,[],[],False,NaN,4.0,8,"['5', '15', '2018']",ruTcyqV1E7UlOZ4I2ZLJHQ,en,"Pulled pork burger, salmon and cream cheese ba...","[1, 0, 0]",Qr5QmYRZHrh7zaqjYw63AQ,"Toronto, Canada",1.0,False,2018,15,5,1902,8511,8511,black-goat-cafe-toronto,"{'latitude': 43.59672, 'longitude': -79.52254}",+1 416-792-7200,5179.05848,https://s3-media2.fl.yelpcdn.com/bphoto/QWleI6...,False,"{'address1': '3261 Lake Shore Boulevard W', 'a...",Black Goat Cafe,1.416793e+10,$,3.5,17,[],https://www.yelp.com/biz/black-goat-cafe-toron...,43.59672,-79.52254,2018-05-15,Cafes


In [55]:
print('#of reviews:', len(df))

#of reviews: 402927


In [56]:
df_business_clean.head(2)

,level_0,index,Unnamed: 0,alias,categories,coordinates,display_phone,distance,business_id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,latitude,longitude
0,0,0,0,the-ossington-toronto,"[{'alias': 'divebars', 'title': 'Dive Bars'}]","{'latitude': 43.6456, 'longitude': -79.41903}",+1 416-850-0161,4831.221092,KUdj08tVC3tvIT0rM6V8uQ,https://s3-media1.fl.yelpcdn.com/bphoto/dEQPAb...,False,"{'address1': '61 Ossington Avenue', 'address2'...",The Ossington,1.416850e+10,$$,3.0,46,[],https://www.yelp.com/biz/the-ossington-toronto...,43.6456,-79.41903
1,1,1,1,life-is-sweet-toronto-3,"[{'alias': 'desserts', 'title': 'Desserts'}, {...","{'latitude': 43.67212500034, 'longitude': -79....",+1 416-698-0555,1716.906703,rnjFHvb1I-KlnGvv_o_Zxg,https://s3-media2.fl.yelpcdn.com/bphoto/s11QxZ...,False,"{'address1': '2345 Queen Street E', 'address2'...",Life is Sweet,1.416698e+10,$$,4.0,10,[],https://www.yelp.com/biz/life-is-sweet-toronto...,43.67212500034,-79.28946080056


In [57]:
set(df['categories'])

{'Coffee & Tea, Reunion, Coffee Roasteries',
 'Italian, Pizza, Cafes',
 'Coffee & Tea, Juice Bars & Smoothies, Sandwiches',
 'Desserts, Belgian, Ice Cream & Frozen Yogurt',
 'Ramen, Desserts',
 'Japanese, Korean, Asian Fusion',
 'Fast Food, Poutineries',
 'Bakeries, Bagels, Cafes',
 'Bakeries, Filipino',
 'Coffee & Tea, Bakeries, Breakfast & Brunch',
 'Canadian (New), American (Traditional), Fast Food',
 'Food Trucks, Mexican',
 'Dim Sum, Sushi Bars',
 'Italian, Tapas/Small Plates',
 'Cafes, Salad, Soup',
 'Poutineries, Sandwiches',
 'Indian, Buffets, Pakistani',
 'Indian, African',
 'Vegetarian, Juice Bars & Smoothies',
 'Pizza, Italian, Salad',
 'Malaysian, Chinese, Asian Fusion',
 'Desserts, Food Court',
 'Jazz & Blues, Tapas/Small Plates',
 'Sushi Bars, Japanese, Buffets',
 'Sports Bars, Burgers, Beer Bar',
 'Venues & Event Spaces, Russian',
 'Chinese, Soup',
 'Chinese, Noodles, Fast Food',
 'Soup',
 'Seafood, Fish & Chips, British',
 'Chicken Shop, Fast Food',
 'Bakeries, Desserts

## Extract distinct categories

In [58]:
final_category = list(df['categories'])

In [59]:
aList = []
for categories in final_category:
    split_category = [cat.strip() for cat in categories.split(',')]
    for category in split_category:
        aList.append(category)

# Exporting data

In [60]:
Export = df.to_json(r'../data/Cleaned_Toronto_Reviews.json', orient='records')

In [61]:
Export = df_business_clean.to_json(r'../data/Cleaned_Toronto_Business.json', orient='records')

# Checking the export data

In [142]:
filename = "..\\data\\Cleaned_Toronto_Business.json"
with open(filename,'r') as f:
        data = f.readlines()
        data = list(map(json.loads, data))
    
data = data[0]
#Get all the data from the dggeata file
df = pd.DataFrame(data)

df.rename(columns={'stars': 'review_stars', 'text': 'review_text', 'cool': 'review_cool',
           'funny': 'review_funny', 'useful': 'review_useful'}, inplace=True)

In [143]:
df.head(5)

,Unnamed: 0,alias,business_id,categories,coordinates,display_phone,distance,image_url,index,is_closed,latitude,level_0,location,longitude,name,phone,price,rating,review_count,transactions,url
0,0,the-ossington-toronto,KUdj08tVC3tvIT0rM6V8uQ,"[{'alias': 'divebars', 'title': 'Dive Bars'}]","{'latitude': 43.6456, 'longitude': -79.41903}",+1 416-850-0161,4831.221092,https://s3-media1.fl.yelpcdn.com/bphoto/dEQPAbThPHKevj07OijSfA/o.jpg,0,False,43.6456,0,"{'address1': '61 Ossington Avenue', 'address2': '', 'address3': '', 'city': 'Toronto',...",-79.41903,The Ossington,1.416850e+10,$$,3.0,46,[],https://www.yelp.com/biz/the-ossington-toronto?adjust_creative=uxtIWjm7I33oX3CDHUtQgQ&...
1,1,life-is-sweet-toronto-3,rnjFHvb1I-KlnGvv_o_Zxg,"[{'alias': 'desserts', 'title': 'Desserts'}, {'alias': 'bakeries', 'title': 'Bakeries'}]","{'latitude': 43.67212500034, 'longitude': -79.28946080056}",+1 416-698-0555,1716.906703,https://s3-media2.fl.yelpcdn.com/bphoto/s11QxZr-V8-VJHNLQJyvgQ/o.jpg,1,False,43.67212500034,1,"{'address1': '2345 Queen Street E', 'address2': '', 'address3': '', 'city': 'Toronto',...",-79.28946080056,Life is Sweet,1.416698e+10,$$,4.0,10,[],https://www.yelp.com/biz/life-is-sweet-toronto-3?adjust_creative=9LfdvYfx0rVPnDmG5bc0i...
2,2,subway-toronto-51,f8YPC7K4UUTkKatE5ga8Ig,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}, {'alias': 'hotdogs', 'title': 'Fast F...","{'latitude': 43.66449, 'longitude': -79.34116}",+1 416-465-8100,2934.260861,https://s3-media1.fl.yelpcdn.com/bphoto/JpxU9FOgghpQQptpdsDDtg/o.jpg,2,False,43.66449,2,"{'address1': '331 Carlaw Avenue', 'address2': 'Unit 103', 'address3': '', 'city': 'Tor...",-79.34116,Subway,1.416466e+10,$$,1.5,6,[],https://www.yelp.com/biz/subway-toronto-51?adjust_creative=NeTgDT5IUqTFt2XAEV__OA&utm_...
3,5,jodies-restaurant-toronto,K5TteBnAfgj18HRnV-Yy4w,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {'alias': 'diners', 'title': 'Diners'}, {...","{'latitude': 43.6053352355957, 'longitude': -79.5289154052734}",+1 416-255-5552,4262.777626,https://s3-media3.fl.yelpcdn.com/bphoto/GtkSIOchBTNry8WbIiZ7Gg/o.jpg,5,False,43.6053352355957,3,"{'address1': '331 Horner Avenue', 'address2': '', 'address3': '', 'city': 'Toronto', '...",-79.5289154052734,Jodie's Restaurant,1.416256e+10,$$,4.5,6,[],https://www.yelp.com/biz/jodies-restaurant-toronto?adjust_creative=uxtIWjm7I33oX3CDHUt...
4,11,the-great-canadian-bagel-ltd-vaughan,ARwwcafc0x5s9wC-Te9yXg,"[{'alias': 'bagels', 'title': 'Bagels'}]","{'latitude': 43.7928079, 'longitude': -79.5312639}",+1 905-760-9955,7595.423171,None,11,False,43.7928079,4,"{'address1': '3300 Hwy 7', 'address2': 'Suite 101', 'address3': '', 'city': 'Vaughan',...",-79.5312639,"The Great Canadian Bagel, Ltd",1.905761e+10,$$,4.0,1,[],https://www.yelp.com/biz/the-great-canadian-bagel-ltd-vaughan?adjust_creative=9LfdvYfx...


In [ ]:
pd.set_option('display.max_columns', None)

In [144]:
set(df.categories.values)

{"[{'alias': 'hotdog', 'title': 'Hot Dogs'}, {'alias': 'sportsbars', 'title': 'Sports Bars'}, {'alias': 'beerbar', 'title': 'Beer Bar'}]",
 "[{'alias': 'italian', 'title': 'Italian'}, {'alias': 'pizza', 'title': 'Pizza'}, {'alias': 'sandwiches', 'title': 'Sandwiches'}]",
 "[{'alias': 'pizza', 'title': 'Pizza'}, {'alias': 'italian', 'title': 'Italian'}, {'alias': 'sandwiches', 'title': 'Sandwiches'}]",
 "[{'alias': 'pakistani', 'title': 'Pakistani'}, {'alias': 'bbq', 'title': 'Barbeque'}, {'alias': 'seafood', 'title': 'Seafood'}]",
 "[{'alias': 'cocktailbars', 'title': 'Cocktail Bars'}]",
 "[{'alias': 'coffee', 'title': 'Coffee & Tea'}, {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}]",
 "[{'alias': 'filipino', 'title': 'Filipino'}, {'alias': 'desserts', 'title': 'Desserts'}, {'alias': 'popupshops', 'title': 'Pop-up Shops'}]",
 "[{'alias': 'personalchefs', 'title': 'Personal Chefs'}, {'alias': 'catering', 'title': 'Caterers'}]",
 "[{'alias': 'mediterranean', 'title': 'Medit

In [60]:
df[df['categories'].str.contains('Food Trucks')].categories.values

array(["[{'alias': 'venezuelan', 'title': 'Venezuelan'}, {'alias': 'foodtrucks', 'title': 'Food Trucks'}]",
       "[{'alias': 'icecream', 'title': 'Ice Cream & Frozen Yogurt'}, {'alias': 'foodtrucks', 'title': 'Food Trucks'}]",
       "[{'alias': 'foodtrucks', 'title': 'Food Trucks'}]",
       "[{'alias': 'southern', 'title': 'Southern'}, {'alias': 'foodtrucks', 'title': 'Food Trucks'}]",
       "[{'alias': 'foodtrucks', 'title': 'Food Trucks'}, {'alias': 'japanese', 'title': 'Japanese'}, {'alias': 'asianfusion', 'title': 'Asian Fusion'}]",
       "[{'alias': 'bbq', 'title': 'Barbeque'}, {'alias': 'foodtrucks', 'title': 'Food Trucks'}]",
       "[{'alias': 'gourmet', 'title': 'Specialty Food'}, {'alias': 'foodtrucks', 'title': 'Food Trucks'}]",
       "[{'alias': 'foodtrucks', 'title': 'Food Trucks'}, {'alias': 'vegan', 'title': 'Vegan'}]",
       "[{'alias': 'catering', 'title': 'Caterers'}, {'alias': 'foodtrucks', 'title': 'Food Trucks'}]",
       "[{'alias': 'foodtrucks', 'title': 

In [453]:
len(df[df['longitude'] == 'None'])

0

In [148]:
filename = "..\\data\\Cleaned_Toronto_Reviews.json"
with open(filename,'r') as f:
        data = f.readlines()
        data = list(map(json.loads, data))
    
data = data[0]
#Get all the data from the dggeata file
df_reviews = pd.DataFrame(data)

In [149]:
df_reviews.head(2)

,Day,Month,Unnamed: 0.1,Unnamed: 0_x,Unnamed: 0_y,Updated,Year,alias,business_id,business_stars,categories,coordinates,date,display_phone,distance,friend_count,ghost,image_url,img_dsc,img_url,index,is_closed,latitude,level_0,location,longitude,name,nr,phone,photo_count,price,review_count_x,review_count_y,review_date,review_id,review_language,stars,text,transactions,ufc,url,user_id,user_loc,vote_count
0,23,8,6,6,21653,False,2016,happy-lemon-markham,Xo1LNzhnwE-ilqsM3ybs9Q,3.5,"Bubble Tea, Coffee & Tea","{'latitude': 43.8258200636421, 'longitude': -79.3063458055258}",2016-08-23,+1 416-855-0768,2867.240253,105,False,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv5Siz8ExZpAC8XSXA/o.jpg,"['Photo of Happy Lemon - Markham, ON, Canada. Lemon mango slush']",['https://s3-media4.fl.yelpcdn.com/bphoto/tu7jZIEA1RZJ6U1s7ZAx3w/348s.jpg'],21653,False,43.8258200636421,5749,"{'address1': 'B25 4300 Steeles Avenue E', 'address2': '', 'address3': None, 'city': 'M...",-79.3063458055258,Happy Lemon,False,1.416855e+10,325.0,$,122,32,"['8', '23', '2016']",rqjzFEb1rQ09CgDVlGxZug,en,4.0,I ordered the lemon mango slush and the lemon taste is very strong. If you love lemon...,[],"[1, 1, 1]",https://www.yelp.com/biz/happy-lemon-markham?adjust_creative=ZbO96qTvvfSmFTkvkUBjkQ&ut...,zsJFjhBQEFQ6gJ7BsNM_Ug,"Toronto, Canada",1.0
1,2,10,7,7,21653,False,2016,happy-lemon-markham,Xo1LNzhnwE-ilqsM3ybs9Q,3.5,"Bubble Tea, Coffee & Tea","{'latitude': 43.8258200636421, 'longitude': -79.3063458055258}",2016-10-02,+1 416-855-0768,2867.240253,171,False,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv5Siz8ExZpAC8XSXA/o.jpg,"['Photo of Happy Lemon - Markham, ON, Canada. Oreo milk tea', 'Photo of Happy Lemon - ...","['https://s3-media3.fl.yelpcdn.com/bphoto/h110aP3Vldr7KjuhOhryZQ/348s.jpg', 'https://s...",21653,False,43.8258200636421,5749,"{'address1': 'B25 4300 Steeles Avenue E', 'address2': '', 'address3': None, 'city': 'M...",-79.3063458055258,Happy Lemon,False,1.416855e+10,307.0,$,114,32,"['10', '2', '2016']",P7S5VclOJEihkqpA7LscPw,en,3.0,"Came here on a Sunday afternoon, it wasn't busy at all. I came here for the Sunday spe...",[],"[1, 0, 0]",https://www.yelp.com/biz/happy-lemon-markham?adjust_creative=ZbO96qTvvfSmFTkvkUBjkQ&ut...,P7YuMh74-I2cDq7oU8frww,"York Regional Municipality, Canada",1.0
